In [1]:
import pandas as pd
import numpy as np
import statistics
import requests
import json
import operator

## Common Methods

In [2]:
def callApi(method, stock, isAnnual = False ):
    key = "token=" + "pk_ee5af08dfb9a419aaba2cbee77b80165"
    host = "https://cloud.iexapis.com/stable/"
    ticker = "stock/"+ stock
    api_method ="/" + method 
    query = "?"
    add = "&"
    annualLast = "/10"
    annualPeriod = "period=annual"
    quarterLast = "/4"
    quarterPeriod = "period=quarter"
    if isAnnual:
        url = host + ticker + api_method + annualLast + query + annualPeriod + add + key 
    else:
        url = host + ticker + api_method + quarterLast + query + quarterPeriod + add + key 
    try:
        response = requests.request("GET", url)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [3]:
def addTTMToYearLists(method,lists,ttmList):
    reportDate = "reportDate"
    if method == "earnings":
        reportDate = "EPSReportDate"
    if not (method in ttmList.keys()):
        return lists
    first = lists[method][0]
    ttm = ttmList[method][0]
    if first[reportDate] != ttm[reportDate]: 
        lists[method].insert(0, ttmList[method][0])
    return lists

## Ratio Methods

In [4]:
def peRatio(quote):
    if "peRatio" in quote.keys() and quote["peRatio"] is not None:
        return float(quote["peRatio"])
    return 0.0

In [5]:
def priceToBook(stats):
    if "priceToBook" in stats.keys() and stats["priceToBook"] is not None:
        return float(stats["priceToBook"])
    return 0.0

In [6]:
def debtToEquity(stats):
    if "debtToEquity" in stats.keys() and stats["debtToEquity"] is not None:
        return float(stats["debtToEquity"])
    return 0.0 

In [44]:
def pegRatio(stats):
    if"pegRatio" in stats.keys() and stats["pegRatio"] is not None:
        return float(stats["pegRatio"])
    return 0.0

In [9]:
def marketCap(quote):
    if "marketCap" in quote.keys() and quote["marketCap"] is not None:
        return quote["marketCap"]
    return 0 

In [10]:
# Current Ratio= Current Assets / Current Liabilities
def currentRatio(balance_sheet):
    if "balancesheet" in balance_sheet.keys() and len(balance_sheet["balancesheet"]) > 0:
        bs = balance_sheet["balancesheet"][0]
        currentAssets = bs['currentAssets']
        totalCurrentLiabilities = bs['totalCurrentLiabilities']
        return currentAssets / totalCurrentLiabilities
    return 0                                                                                                   

In [35]:
def checkErningProgression(earnings, earning):
    earnings = addTTMToYearLists("earnings", earnings, earning)
    if not("earnings" in earnings.keys()) or len(earnings["earnings"]) == 0:
        return False
    earningsSet = earnings["earnings"][:-1]
    yearAgoChangePercent = np.array([])
    for earning in earningsSet:
        if not("yearAgoChangePercent" in earning.keys()):
            return False
        if earning["yearAgoChangePercent"] is not None:
            yearAgoChangePercent = np.append(yearAgoChangePercent, int(earning["yearAgoChangePercent"]))
    if len(yearAgoChangePercent) == 0 :
        return False
    isDecending = (yearAgoChangePercent == np.sort(yearAgoChangePercent)[::-1]).all()
    return isDecending

In [49]:
def sharesOutstanding(stats):
    if "sharesOutstanding" in stats.keys() and stats["sharesOutstanding"] is not None:
        return float(stats["sharesOutstanding"])
    return 0.0

In [13]:
def NetCurrentAssetValuePerShare(balanceSheet, stats, price):
    #Net Current Asset Value = Current Assets – (Current Liabilities + Long-Term Liabilities) 
    shares_Outstanding = sharesOutstanding(stats)
    if "balancesheet" in balance_sheet.keys() and len(balance_sheet["balancesheet"]) > 0 and shares_Outstanding !=0:
        latest_balance_sheet = balance_sheet["balancesheet"][0]
        if "currentAssets" in latest_balance_sheet.keys() and "totalLiabilities" in latest_balance_sheet.keys() and "minorityInterest" in latest_balance_sheet.keys():
            netCurrentAssetValue = (latest_balance_sheet["currentAssets"] - latest_balance_sheet["totalLiabilities"] - latest_balance_sheet["minorityInterest"]) / shares_Outstanding
            return price/netCurrentAssetValue
    return 0

In [14]:
# Last Six moths data 
def insideTrade(insiderSummary):
    return sum(item['netTransacted'] for item in insiderSummary)

In [75]:
def ZScore(balance_sheet, incomes, quote):

    X1 = X2 = X3 = X4 = X5 = 0
    
    if "balancesheet" in balance_sheet.keys() and len(balance_sheet["balancesheet"]) > 0 and "income" in income.keys() and len(income["income"]) > 0:
        latest_balance_sheet = balance_sheet["balancesheet"][0]
        latest_income = income["income"][0]
        
        #X1 = Working Capital / Total Assets
        # =(Total Current Assets - Total Current Liabilities) / Total Assets
        if "currentAssets" in latest_balance_sheet.keys() and "totalCurrentLiabilities" in latest_balance_sheet.keys() and "totalAssets" in latest_balance_sheet.keys():
            totalCurrentAssets = latest_balance_sheet["currentAssets"]
            totalLiabilities = latest_balance_sheet["totalCurrentLiabilities"]
            totalAssets = latest_balance_sheet["totalAssets"]
            if totalCurrentAssets is not None and totalLiabilities is not None and totalAssets is not None: 
                X1 = (totalCurrentAssets - totalLiabilities) /  totalAssets
        
        #X2 = Retained Earnings /Total Assets    
        if "retainedEarnings" in latest_balance_sheet.keys() and "totalAssets" in latest_balance_sheet.keys():
            retainedEarnings = latest_balance_sheet["retainedEarnings"]
            totalAssets = latest_balance_sheet["totalAssets"]
            if retainedEarnings is not None and totalAssets is not None:
                X2 = retainedEarnings /  totalAssets
        
        #X3 = Earnings Before Interest and Taxes / Total Assets
        #    = (Pre-Tax Income - Interest Expense) / Total Assets
        # Not able to get Interest Expense
        if "pretaxIncome" in latest_income.keys() and "totalAssets" in latest_balance_sheet.keys():
            pretaxIncome = sum(income['pretaxIncome'] for income in incomes["income"])
            totalAssets = latest_balance_sheet["totalAssets"]
            if pretaxIncome is not None and totalAssets is not None:
                X3 = pretaxIncome / totalAssets
        
        
        #X4 = Market Value Equity / Book Value of Total Liabilities
        if "totalLiabilities" in latest_balance_sheet.keys():
            market_value_equity = marketCap(quote)
            totalLiabilities = latest_balance_sheet['totalLiabilities']
            if market_value_equity is not None and totalLiabilities is not None:
                X4 = pretaxIncome/ totalAssets
            
        #X5 = Revenue / Total Assets
        if "income" in income.keys() and "totalAssets" in latest_balance_sheet.keys():
            totalRevenue = sum(income['totalRevenue'] for income in incomes["income"])
            totalAssets = latest_balance_sheet["totalAssets"]
            if totalRevenue is not None and totalAssets is not None:
                X5 = totalRevenue / totalAssets
    print(X1)
    print(X2)
    print(X3)
    print(X4)
    print(X5)

    zScore = 1.2 * X1 + 1.4 * X2 + 3.3 * X3 + 0.6 * X4 + 1.0 * X5
        
        #ZScore= 1.2* X1 + 1.4 * X2 + 3.3 * X3 + 0.6 * X4 + 1.0 * X5
    return zScore

In [16]:
def FScore(income, cashflows):
    score = 0
    if "income" in income.keys():
        netIncome = sum(net['netIncome'] for net in income["income"])
        if netIncome > 0:
            score += 1 
    if "cashflow" in cashflows.keys():
        cashflow = sum(flow['cashFlow'] for flow in cashflows["cashflow"]) 
        if cashflow > 0:
            score += 1 
    
    return 0 

In [76]:

tickers = ['AAPL']
dataSet = [] 
for ticker in tickers:
    data = {}
    points = 0
    
    quote = callApi("quote", ticker)
    
    # Less than 10 million don't comsider
    market_cap =  marketCap(quote)
    if market_cap < 10000000:
        continue
        
    # Z-Score 
    balance_sheet = callApi("balance-sheet", ticker)
    income = callApi("income", ticker)
    zScore = ZScore(balance_sheet, income, quote)
    if zScore is not None :
        if zScore > 7.99:
            points += 2
        elif zScore > 2.99:
            ponts += 1
        else: 
            continue
    
    # Major data set
    company = callApi("company", ticker)
    stats = callApi("advanced-stats", ticker)
    earnings = callApi("earnings", ticker, True)
    earning = callApi("earnings", ticker)
    price = callApi("price", ticker)
    inside_trade = callApi("insider-summary", ticker)
    
    
    data["Ticker"] = ticker
    data ["company Name"] = quote["companyName"]
    data["Sector"] = company["sector"]
    data["Price"] = price
    data["Market Cap"] = market_cap
    data["52 High"] = quote["week52High"]
    data["52 Low"] = quote["week52Low"]
    data["Z Score"] =  zScore
    
    # PE Ratio < 7
    pe_ratio = peRatio(quote)
    if pe_ratio != 0: 
        data["PE Ratio < 7"] = pe_ratio
    else: 
        data["PE Ratio < 7"] = "NA"
    if pe_ratio < 7 and pe_ratio > 0: 
        points += 1
        
    # Price To Book Ratio  
    price_to_book = priceToBook(stats)
    if price_to_book != 0: 
        data["Price To Book < 1"] = price_to_book
    else: 
        data["Price To Book < 1"] = "NA"
    
    if  price_to_book < 1 and  price_to_book > 0: 
        points += 1
    
    # PEG Ratio 
    peg_ratio = pegRatio(stats)
    if peg_ratio != 0: 
        data["PEG Ratio < 1"] = peg_ratio
    else: 
        data["PEG Ratio < 1"] = "NA"
    if peg_ratio < 1.1 and peg_ratio > 0:
        points += 1
    
    # Debt To Equity Ratio 
    # Warent buffet say < 0.5
    debt_Equity = debtToEquity(stats)
    if debt_Equity != 0: 
        data["Debt to Equity < 0.5"] = debt_Equity
    else: 
        data["Debt to Equity < 0.5"] = "NA"
    if debt_Equity < 0.5 and debt_Equity > 0:
        points += 1
        
    # Current Ratio 
    current_ratio = currentRatio(balance_sheet)
    if current_ratio != 0: 
        data["Current Ratio > 3"] = current_ratio
    else: 
        data["Current Ratio > 3"] = "NA"
        
    if current_ratio > 3:
        points += 1
    
    # Earning Progression Last 4 year 
    isProgressing = checkErningProgression(earnings, earning)
    data["Earning Progression"] =  isProgressing
    if isProgressing:
        points += 1
    
    # Net Current Asset Value Per Share
    # Graham instituted a rule in which he would only consider buying a stock if its current price was trading at lower than 66% of its NCAVPS.
    NCAVPS = NetCurrentAssetValuePerShare(balance_sheet, stats, price)
    data["NCAVPS <50%"] =  round(NCAVPS,3)
    if NCAVPS < .50 and NCAVPS > 0:
        points += 1
        
    # Inside Traide
    trade = insideTrade(inside_trade)
    data["Inside Trade"] =  trade
    if trade > 10000:
        points += 1
    
        
    data["Points(10)"] = points

    dataSet.append(data.copy())
dataSet.sort(key=operator.itemgetter("Points(10)"), reverse=True)
df = pd.DataFrame(dataSet)  
df.to_csv("Advance-reserch.csv", sep='\t', encoding='utf-8')
pd.DataFrame(dataSet) 

0.14100471412725624
0.07605626701623475
0.21527742764949076
0.21527742764949076
0.861588686094585


""
